In [2]:
import torch
import torch.nn as nn
import torch.optim as optim



In [6]:
##Bradley TErry Model

import torch
import torch.nn as nn
import torch.optim as optim
class BTModel(nn.Module):
    def __init__(self, N):
        super(BTModel, self).__init__()
        self.reward = nn.Parameter(torch.ones(N))
        
    def forward_exp(self, chosen_id, rejected_id):
        reward_chosen = torch.exp(self.reward[chosen_id])
        reward_rejected = torch.exp(self.reward[rejected_id])
        return reward_chosen / (reward_chosen + reward_rejected)

    def forward_sigmoid(self, chosen_id, rejected_id):
        reward_chosen = self.reward[chosen_id]
        reward_rejected = self.reward[rejected_id]
        return torch.sigmoid(reward_chosen - reward_rejected)

    def loss(self, pred, label):
        return -torch.log(pred) if label == 1 else -torch.log(1 - pred)

In [7]:
N=5
model = BTModel(N)
datas = [(0, 1, 1), (2, 3, 1), (1, 3, 1)]
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [9]:
loss_fn = nn.BCELoss()
for i in range(100):
    total_loss = 0
    for data in datas:
        id_i, id_j, label = data
        optimizer.zero_grad()
        pred = model.forward_sigmoid(id_i, id_j)
        loss = model.loss(pred, torch.tensor(label, dtype=torch.float32))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if i%10==0 : print(f"Epoch {i}, Loss: {total_loss}")

print(model.reward)

Epoch 0, Loss: 2.079441547393799
Epoch 10, Loss: 1.937548816204071
Epoch 20, Loss: 1.811079204082489
Epoch 30, Loss: 1.6980656385421753
Epoch 40, Loss: 1.5967631340026855
Epoch 50, Loss: 1.5056480765342712
Epoch 60, Loss: 1.4234036803245544
Epoch 70, Loss: 1.3488986492156982
Epoch 80, Loss: 1.281165212392807
Epoch 90, Loss: 1.2193749248981476
Parameter containing:
tensor([1.4402, 0.9630, 1.3558, 0.2410, 1.0000], requires_grad=True)


In [ ]:
import torch
import torch.nn.functional as F
from transformers import LlamaConfig, LlamaForCausalLM
torch.manual_seed(42)
# 加载模型
config = LlamaConfig(vocab_size = 32,      
                    hidden_size = 256,
                    intermediate_size = 512,
                    num_hidden_layers = 2,
                    num_attention_heads = 4,
                    num_key_value_heads = 4,
                    )

ref_model = LlamaForCausalLM(config)
ref_model.eval()
model = LlamaForCausalLM(config) 
#print(model.lm_head)

In [11]:
prompt_length = 6
answer_length = 4
prompt_chosen =   torch.tensor([[5, 8, 9, 10, 5, 3,   16, 29, 18, 17]], dtype=torch.int64)
prompt_rejected = torch.tensor([[5, 8, 9, 10, 5, 3,   26, 14, 31, 0]], dtype=torch.int64)
attention_mask =  torch.tensor([[1, 1, 1, 1,  1, 1,   1,  1,  1,  1]], dtype=torch.bool)
labels =  torch.tensor([[0, 0, 0, 0,  0, 0,   1,  1,  1,  1]], dtype=torch.bool)

x_chosen = {'input_ids':prompt_chosen, 'attention_mask':attention_mask}
x_rejected = {'input_ids':prompt_rejected, 'attention_mask':attention_mask}

In [13]:
# test for get logits and logprob
output = ref_model(**x_chosen)

def get_probs(logits, labels):
    per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, 
                                   index=labels.unsqueeze(2)).squeeze(2)
    return per_token_logps

probs_chosen = get_probs(output.logits, prompt_chosen)

print('logits :\n', output.logits.shape)
print('chosen:\n', prompt_chosen[0,-1])
print('chosen:\n', output.logits[0,-1,:])
print('chosen logprob\n', output.logits[0,-1,:].log_softmax(-1))

logits :
 torch.Size([1, 10, 32])
chosen:
 tensor(17)
chosen:
 tensor([ 0.2971,  0.2909,  0.2621,  0.5977, -0.0441, -0.3005,  0.0452,  0.2285,
         0.2922,  0.6789, -0.2369, -0.0842, -0.2411, -0.0244, -0.4583, -0.1332,
        -0.2056, -0.2223,  0.3888, -0.1832,  0.1843, -0.2905,  0.7456, -0.2058,
         0.1334, -0.0196,  0.0427,  0.5138, -0.2301,  0.0559, -0.2310,  0.0939],
       grad_fn=<SliceBackward0>)
chosen logprob
 tensor([-3.2715, -3.2777, -3.3066, -2.9709, -3.6127, -3.8691, -3.5234, -3.3401,
        -3.2764, -2.8898, -3.8055, -3.6528, -3.8097, -3.5930, -4.0269, -3.7018,
        -3.7742, -3.7910, -3.1798, -3.7519, -3.3843, -3.8591, -2.8231, -3.7744,
        -3.4352, -3.5882, -3.5259, -3.0548, -3.7988, -3.5127, -3.7996, -3.4747],
       grad_fn=<LogSoftmaxBackward0>)


In [14]:
import torch.nn.functional as F
#ref/model, chosen/rejected,  logtis/prob value
logits_chosen_ref = ref_model(**x_chosen).logits
logits_rejected_ref = ref_model(**x_rejected).logits
logits_chosen = model(**x_chosen).logits
logits_rejected = model(**x_rejected).logits

probs_chosen_ref = get_probs(logits_chosen_ref, prompt_chosen)
probs_chosen = get_probs(logits_chosen, prompt_chosen)
probs_rejected_ref = get_probs(logits_rejected_ref, prompt_rejected)
probs_rejected = get_probs(logits_rejected, prompt_rejected)

beta = 0.1
pi_logratios = probs_chosen - probs_rejected
ref_logratios = probs_chosen_ref - probs_rejected_ref
logits = pi_logratios - ref_logratios
losses = -F.logsigmoid(beta * logits ) * labels
print(losses)
loss = losses.sum(-1)/attention_mask.sum()
print(loss)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7197, 0.7307, 0.7604,
         0.6830]], grad_fn=<MulBackward0>)
tensor([0.2894], grad_fn=<DivBackward0>)


In [15]:
import torch.optim as optim

model = LlamaForCausalLM(config)
optimizer = optim.SGD(model.parameters(), lr=0.1)
epochs = 1000
epochs_print = epochs//10

neg_poilicy_prob = [] # pi_l
logistic_prob = [] # DPO beta( log(pi_w/pi_ref_w)  - log(pi_l/pi_ref_l))
loss_record = [] # DPO loss
for i in range(epochs):
    
    optimizer.zero_grad()

    # forward get logits
    with torch.no_grad():
        logits_chosen_ref = ref_model(**x_chosen).logits
        logits_rejected_ref = ref_model(**x_rejected).logits
    logits_chosen = model(**x_chosen).logits
    logits_rejected = model(**x_rejected).logits

    # logits to logprob
    probs_chosen_ref = get_probs(logits_chosen_ref, prompt_chosen)
    probs_chosen = get_probs(logits_chosen, prompt_chosen)
    probs_rejected_ref = get_probs(logits_rejected_ref, prompt_rejected)
    probs_rejected = get_probs(logits_rejected, prompt_rejected)

    # loss
    beta = 0.1
    pi_logratios = probs_chosen - probs_rejected
    ref_logratios = probs_chosen_ref - probs_rejected_ref
    logits = pi_logratios - ref_logratios
    losses = -F.logsigmoid( beta * logits ) * label
    loss = losses.sum(-1)/attention_mask.sum()

    # print(loss)
    loss_record.append(loss.item())

    # loss back
    loss.backward()
    optimizer.step()

    neg_poilicy_prob.append(torch.exp(probs_rejected[:,-1]).item())
    logistic_prob.append(torch.sigmoid( beta * logits)[:,-1].item())
    
    if i % epochs_print == 0:
        print(f'step {i}, loss:{loss.item()}, pi_rej:{neg_poilicy_prob[-1]}, log_prob:{logistic_prob[-1]}')
     

step 0, loss:0.6931301951408386, pi_rej:0.023953042924404144, log_prob:0.5267601013183594
step 100, loss:0.4571150243282318, pi_rej:2.137617444164519e-10, log_prob:0.9052523970603943
step 200, loss:0.4340072274208069, pi_rej:4.9613572527650723e-14, log_prob:0.9568975567817688
step 300, loss:0.4272170662879944, pi_rej:4.42454224503625e-16, log_prob:0.9727272391319275
step 400, loss:0.4240449368953705, pi_rej:1.653651504140485e-17, log_prob:0.9802381992340088
step 500, loss:0.4222238063812256, pi_rej:1.3275752523108562e-18, log_prob:0.9845870733261108
step 600, loss:0.4210476875305176, pi_rej:1.7141926205286937e-19, log_prob:0.9874109625816345
step 700, loss:0.4202280640602112, pi_rej:3.060158553549047e-20, log_prob:0.9893866777420044
step 800, loss:0.4196254312992096, pi_rej:6.90988959767223e-21, log_prob:0.990843653678894
step 900, loss:0.4191643297672272, pi_rej:1.8646589711505915e-21, log_prob:0.9919609427452087
